In [20]:
import numpy as np
import pandas
import time
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler

def tdel(row):
    t2 = datetime.strptime(row[0].split(" ")[1], "%H:%M:%S")
    t1 = datetime.strptime('2016-08-25 15:30:00'.split(" ")[1], "%H:%M:%S") #TODO refactor so not hardcoded
    delt = t2 - t1
    return delt.seconds/60    

def tdow(row):
    dow_date = datetime.strptime(row[0].split(" ")[0], "%Y-%m-%d").weekday()
    return dow_date
    
def tdom(row):
    dom_date = datetime.strptime(row[0].split(" ")[0], "%Y-%m-%d").day
    return dom_date

beginning = datetime.now().replace(microsecond=0)

scaler = MinMaxScaler(feature_range=(0, 1))

float_formatter = lambda x: "%.2f" % x
np.set_printoptions(formatter={'float_kind':float_formatter})

r_test = pandas.read_csv('spy_81d.csv', usecols=[0,1,2,3,4,5])

snapdf0 = r_test.copy()

print("# quotes in CSV datafile: ",len(r_test) - 1)

r_test['dt'] = r_test.apply(tdel, axis=1)
r_test['dow'] = r_test.apply(tdow, axis=1)
r_test['dom'] = r_test.apply(tdom, axis=1)

#snapdf2 = r_test.copy()

r_test = r_test.query('dt%2 == 0')
print("Length r_test: ", len(r_test))

r_test = r_test.reset_index()
r_test = r_test.drop('index', 1)

#snapdf_new = r_test.copy()

dates = set()
for index, row in r_test.iterrows():
        dates.add(row[0].split(" ")[0])

r_test[['o', 'h', 'l', 'c', 'v', 'dt', 'dow', 'dom']] = scaler.fit_transform(r_test[['o', 'h', 'l', 'c', 'v', 'dt', 'dow', 'dom']])        

dates_l = sorted(list(dates))

print("# of dates: ", len(sorted(dates_l)))

window = 60
forecast = 1

# remember (z,y,z) where z = 5 + number of extra features
samples = np.ndarray((0,window,8), dtype = float)
results = np.ndarray((0,), dtype = float)
    
for date in dates_l:
    min = 0
    timing_now = datetime.now().replace(microsecond=0) - beginning
    print("Date:", date, "Time in loop:", timing_now)
    r_date = r_test[r_test["ts"].str.contains(date)]
    r_date = r_date.drop('ts', 1)
    
    while len(r_date) > window + min + forecast:
        s  = r_date.values[min : window + min]
        f = r_date.values[window + min + forecast - 1][0]
        l = r_date.values[window + min - 1][0]
        d = f - l

        # remember (z,y,z) where z = 5 + number of extra features
        s = s.reshape(1, window, 8)
        
        if d < 0:
            d = 0
        else:
            d = 1
        
        samples = np.append(samples, s, axis=0)
        results = np.append(results, d)
        min += 1
        
results = results.reshape(-1,1)    
results = scaler.fit_transform(results)    

print("Shape of samples: ", samples.shape, " - Shape of results: ", results.shape)
print(samples[0:1])
print(results[0:1])

ending = datetime.now().replace(microsecond=0)
print(ending - beginning)

# split into train and test sets
train_size = int(len(samples) * 0.67)
test_size = len(samples) - train_size
train_s, test_s = samples[0:train_size,:], samples[train_size:len(samples),:]
train_size = int(len(results) * 0.67)
test_size = len(results) - train_size
train_r, test_r = results[0:train_size,:], results[train_size:len(results),:]

print(train_s.shape)
print(test_s.shape)
print(train_r.shape)
print(test_r.shape)

print(train_s.dtype)
print(test_s.dtype)
print(train_r.dtype)
print(test_r.dtype)

np.save('train_s_81day17may-test-d2', train_s)
np.save('train_r_81day17may-test-d2', train_r)
np.save('test_s_81day17may-test-d2', test_s)
np.save('test_r_81day17may-test-d2', test_r)
print("***** DONE *****")

# quotes in CSV datafile:  31589
Length r_test:  15795
# of dates:  81
Date: 2016-07-28 Time in loop: 0:00:04
Date: 2016-07-29 Time in loop: 0:00:04
Date: 2016-08-01 Time in loop: 0:00:04
Date: 2016-08-02 Time in loop: 0:00:04
Date: 2016-08-03 Time in loop: 0:00:04
Date: 2016-08-04 Time in loop: 0:00:04
Date: 2016-08-05 Time in loop: 0:00:05
Date: 2016-08-08 Time in loop: 0:00:05
Date: 2016-08-09 Time in loop: 0:00:05
Date: 2016-08-10 Time in loop: 0:00:05
Date: 2016-08-11 Time in loop: 0:00:05
Date: 2016-08-12 Time in loop: 0:00:06
Date: 2016-08-15 Time in loop: 0:00:06
Date: 2016-08-16 Time in loop: 0:00:06
Date: 2016-08-17 Time in loop: 0:00:06
Date: 2016-08-18 Time in loop: 0:00:07
Date: 2016-08-19 Time in loop: 0:00:07
Date: 2016-08-22 Time in loop: 0:00:08
Date: 2016-08-23 Time in loop: 0:00:08
Date: 2016-08-24 Time in loop: 0:00:09
Date: 2016-08-25 Time in loop: 0:00:09
Date: 2016-08-26 Time in loop: 0:00:10
Date: 2016-08-29 Time in loop: 0:00:10
Date: 2016-08-30 Time in loop: 0

In [24]:
import json
print(str(snapdf0.head().to_json))
del snapdf0['index']
snapdf0.head().to_json(orient='split')

<bound method NDFrame.to_json of                     ts        o        h        l        c         v
0  2016-07-28 15:30:00  215.185  215.265  215.105  215.255  668765.0
1  2016-07-28 15:31:00  215.255  215.265  215.205  215.235  264511.0
2  2016-07-28 15:32:00  215.245  215.384  215.228  215.379  231390.0
3  2016-07-28 15:33:00  215.379  215.404  215.285  215.290  146140.0
4  2016-07-28 15:34:00  215.285  215.304  215.225  215.235  142023.0>


'{"columns":["ts","o","h","l","c","v"],"index":[0,1,2,3,4],"data":[["2016-07-28 15:30:00",215.185,215.265,215.105,215.255,668765.0],["2016-07-28 15:31:00",215.255,215.265,215.205,215.235,264511.0],["2016-07-28 15:32:00",215.245,215.384,215.228,215.379,231390.0],["2016-07-28 15:33:00",215.379,215.404,215.285,215.29,146140.0],["2016-07-28 15:34:00",215.285,215.304,215.225,215.235,142023.0]]}'

In [ ]:
from datetime import datetime
datetime.strptime('2016.08.25 15:30:00', "%Y.%m.%d %H:%M:%S")